# Libs

In [38]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import scale
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import Perceptron
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [ ]:
# Загрузите выборку Wine по адресу 
# https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data
#  (файл также приложен к этому заданию)

# Извлеките из данных признаки и классы. Класс записан в первом столбце (три варианта), 
# признаки — в столбцах со второго по последний. Более подробно о сути признаков можно прочитать по адресу 
# https://archive.ics.uci.edu/ml/datasets/Wine
#  (см. также файл wine.names, приложенный к заданию)

# Оценку качества необходимо провести методом кросс-валидации по 5 блокам (5-fold). 
# Создайте генератор разбиений, который перемешивает выборку перед формированием блоков (shuffle=True). 
# Для воспроизводимости результата, создавайте генератор KFold с фиксированным параметром random_state=42. 
# В качестве меры качества используйте долю верных ответов (accuracy).

# Найдите точность классификации на кросс-валидации для метода k ближайших соседей 
# (sklearn.neighbors.KNeighborsClassifier), при k от 1 до 50. При каком k получилось оптимальное качество? 
# Чему оно равно (число в интервале от 0 до 1)? Данные результаты и будут ответами на вопросы 1 и 2.

# Произведите масштабирование признаков с помощью функции sklearn.preprocessing.scale. 
# Снова найдите оптимальное k на кросс-валидации.

# Какое значение k получилось оптимальным после приведения признаков к одному масштабу? 
# Приведите ответы на вопросы 3 и 4. Помогло ли масштабирование признаков?

# Если ответом является нецелое число, то целую и дробную часть необходимо разграничивать точкой, например, 0.5. 
# При необходимости округляйте дробную часть до двух знаков.

# Ответ на каждое задание — текстовый файл, содержащий ответ в первой строчке. Обратите внимание, 
# что отправляемые файлы не должны содержать пустую строку в конце. 

# hw1

In [67]:
df = pd.DataFrame(np.genfromtxt('wine.data', delimiter=','))
col = [ 'Class','Alcohol','Malic_acid','Ash','Alcalinity_of_ash',
        'Magnesium','Total_phenols','Flavanoids','Nonflavanoid_phenols',
        'Proanthocyanins','Color_intensity','Hue','OD280/OD315_of_diluted_wines',
        'Proline']
df.columns = col
df.columns = df.columns.str.lower()

In [68]:
X = df.iloc[:,1:]
y = df.iloc[:,0]

In [69]:
df_lst = []
for i in range(1, 51):
    neigh = KNeighborsClassifier(n_neighbors=i)
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(neigh, X, y, cv=kf, scoring='accuracy')
    # print('k:', i, 'Acuracy:', scores.mean())
    df = pd.DataFrame({'k':i, 'accu':scores.mean().round(3)}, index=[0])
    df_lst.append(df)
result = pd.concat(df_lst, axis=0).reset_index()
result.sort_values(by='accu', ascending=False).head(3)

,index,k,accu
0,0,1,0.730
33,0,34,0.725
34,0,35,0.725


In [74]:
X_norm = scale(X)

df_lst2 = []
for i in range(1, 51):
    neigh = KNeighborsClassifier(n_neighbors=i)
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(neigh, X_norm, y, cv=kf, scoring='accuracy')
    # print('k:', i, 'Acuracy:', scores.mean())
    df2 = pd.DataFrame({'k':i, 'accu':scores.mean().round(2)}, index=[0])
    df_lst2.append(df2)
result2 = pd.concat(df_lst2, axis=0).reset_index()
result2.sort_values(by='accu', ascending=False).head(3)

,index,k,accu
28,0,29,0.98
14,0,15,0.97
13,0,14,0.97


In [ ]:
# Мы будем использовать в данном задании набор данных Boston, где нужно предсказать стоимость жилья на основе 
# различных характеристик расположения (загрязненность воздуха, близость к дорогам и т.д.). 
# Подробнее о признаках можно почитать по адресу 
# https://archive.ics.uci.edu/ml/machine-learning-databases/housing/

# Загрузите выборку Boston с помощью функции sklearn.datasets.load_boston(). 
# Результатом вызова данной функции является объект, у которого признаки записаны в поле data, а целевой вектор — в поле target.

# Приведите признаки в выборке к одному масштабу при помощи функции sklearn.preprocessing.scale.
# Переберите разные варианты параметра метрики p по сетке от 1 до 10 с таким шагом, чтобы всего было протестировано 200 вариантов 
# (используйте функцию numpy.linspace). 
# Используйте KNeighborsRegressor с n_neighbors=5 и weights='distance' — данный параметр добавляет в алгоритм веса, 
# зависящие от расстояния до ближайших соседей. В качестве метрики качества используйте среднеквадратичную ошибку 
# (параметр scoring='mean_squared_error' у cross_val_score; при использовании библиотеки scikit-learn версии 0.18.1 и выше 
# необходимо указывать scoring='neg_mean_squared_error').  Качество оценивайте, как и в предыдущем задании, 
# с помощью кросс-валидации по 5 блокам с random_state = 42, не забудьте включить перемешивание выборки (shuffle=True).

# Определите, при каком p качество на кросс-валидации оказалось оптимальным. Обратите внимание, что cross_val_score возвращает 
# массив показателей качества по блокам; необходимо максимизировать среднее этих показателей. 
# Это значение параметра и будет ответом на задачу.

# Если ответом является нецелое число, то целую и дробную часть необходимо разграничивать точкой, например, 0.4. 
# При необходимости округляйте дробную часть до одного знака.

# hw2

In [23]:
sklearn.__version__

'1.4.2'

In [24]:
boston  = pd.read_csv('boston.csv').drop('Unnamed: 0', axis=1)
X = boston.iloc[:, :-1]
y = boston.iloc[:, -1]
X_norm = scale(X)
p_linspace = np.linspace(1, 10, 200)

df_lst3 = []
for p in p_linspace:
    neigh = KNeighborsRegressor(n_neighbors=5, weights='distance', metric='minkowski', p=p)
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(neigh, X_norm, y, cv=kf, scoring='neg_mean_squared_error')
    df3 = pd.DataFrame({'p':p, 'neg_mse':scores.mean().round(2)}, index=[0])
    df_lst3.append(df3)
result3 = pd.concat(df_lst3, axis=0).reset_index(drop=True)
result3.sort_values(by='neg_mse', ascending=False).head(3)

,p,neg_mse
0,1.000000,-16.03
2,1.090452,-16.37
1,1.045226,-16.41


# hw3

In [ ]:
# Загрузите обучающую и тестовую выборки из файлов perceptron-train.csv и perceptron-test.csv. 
# Целевая переменная записана в первом столбце, признаки — во втором и третьем.

# Обучите персептрон со стандартными параметрами и random_state=241.

# Подсчитайте качество (долю правильно классифицированных объектов, accuracy) полученного классификатора на тестовой выборке.

# Нормализуйте обучающую и тестовую выборку с помощью класса StandardScaler. 

# Обучите персептрон на новой выборке. Найдите долю правильных ответов на тестовой выборке.

# Найдите разность между качеством на тестовой выборке после нормализации и качеством до нее. 
# Это число и будет ответом на задание.

# Если ответом является нецелое число, то целую и дробную часть необходимо разграничивать точкой, например, 0.421. 
# При необходимости округляйте дробную часть до трех знаков.

# Ответ на каждое задание — текстовый файл, содержащий ответ в первой строчке. Обратите внимание, 
# что отправляемые файлы не должны содержать перевод строки в конце. Данный нюанс является ограничением платформы Coursera. 
# Мы работаем над тем, чтобы убрать это ограничение.

In [37]:
perc_train = pd.read_csv('perceptron-train.csv', names = ['target', 'feature1', 'feature2'])
perc_test = pd.read_csv('perceptron-test.csv', names = ['target', 'feature1', 'feature2'])
X_train = perc_train.iloc[:, 1:]
y_train = perc_train.iloc[:, 0]
X_test = perc_test.iloc[:, 1:]
y_test = perc_test.iloc[:, 0]

In [78]:
clf = Perceptron(max_iter=5, tol=None, random_state=241)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
accu = accuracy_score(y_test, predictions)
accu

0.655

In [80]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
clf.fit(X_train_scaled, y_train)
predictions_norm = clf.predict(X_test_scaled)
accu_norm = accuracy_score(y_test, predictions_norm)
round(accu_norm - accu,3)

0.185

In [63]:
X_test_scaled
# array([[ 1.67171578e+00,  1.50744899e+00],
#        [-8.32374159e-01,  1.35261123e+00],
#        [ 8.14843791e-01, -4.12493370e-01],
#        [ 2.07920119e-01,  2.16572204e+00],
#        [-4.02605851e-01,  6.93187424e-01],
#        [-1.46865651e+00,  1.07554509e+00],
#        [-1.21763005e+00, -3.83631116e-01],
# array([[ 1.70453471,  1.22307108],
#        [-0.81453227,  1.08650068],
#        [ 0.84253774, -0.47036136],
#        [ 0.23198404,  1.80368275],
#        [-0.38219351,  0.50487403],
#        [-1.45462024,  0.84212211],
#        [-1.20209239, -0.4449042 ],

array([[ 1.70453471,  1.22307108],
       [-0.81453227,  1.08650068],
       [ 0.84253774, -0.47036136],
       [ 0.23198404,  1.80368275],
       [-0.38219351,  0.50487403],
       [-1.45462024,  0.84212211],
       [-1.20209239, -0.4449042 ],
       [-0.34113458, -1.06212649],
       [ 0.52691477, -0.16554488],
       [-0.83087342, -1.70159536],
       [ 0.73273175,  0.32901624],
       [ 0.13161634,  1.51388433],
       [ 0.60645948,  0.32089563],
       [ 0.80656227,  3.30973872],
       [-0.04645073,  0.81016667],
       [ 1.29578352,  1.46447895],
       [-0.33872243, -0.46744992],
       [-0.81346886,  0.34009243],
       [-0.96776119,  0.70468991],
       [-0.30271524, -0.64399419],
       [ 3.01231298, -1.20459761],
       [ 0.48842589, -0.18290443],
       [-0.32699379, -1.21561899],
       [ 0.1767683 ,  0.05234161],
       [ 0.15646125, -1.94100579],
       [-0.16569092,  0.86383418],
       [ 0.63215651, -1.06897633],
       [-0.26254646, -0.76591945],
       [-0.33944277,